## ## Funçao para criar tabela das musicas ate 2015.

In [1]:
import re
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
def criar_tabela_ate_2015(ano):
    
    # definindo o headers e a url
    url_musica = f'https://maistocadas.mus.br/{ano}/'
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    
    # fazendo o http request e tratando erros (se a resposta nao for 200)
    try:
        response = requests.get(url_musica, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request falhou para o ano {ano}: {e}")
        return pd.DataFrame()
    
    # fazendo o parse do html
    soup = BeautifulSoup(response.text, 'html.parser')
    target_div = soup.find('div', {'class': 'lista'})
    if target_div is None:
        print(f'Tabela não encontrada no ano {ano}')
        return pd.DataFrame()
    
    # pre processando o texto
    target_div_text = target_div.text.replace(f"Lista das mais tocadas {ano}", "")
    target_div_text = re.sub(r'(?<=\w) -', ' – ', target_div_text)
    target_div_text = re.sub(r'(?<=\d)\s?–\s?', ' – ', target_div_text)
    
    # utilizando expressao regular para encontrar as musicas, artistas, gravadora e vezes tocada.
    padrao = re.compile(r'(.*?) – (.*?) – Tocou (.*?) vezes(?: \((.*?)\))?')
    encontrados = padrao.finditer(target_div_text)
    
    # processando as musicas encontradas, criando uma lista e passando um dicionario de cada musica para a lista.
    data = []
    for encontrado in encontrados:
        musica, artista, tocada, gravadora = encontrado.groups()
        tocada = str(tocada.replace('.', ''))
        data.append({
            'Música': musica.strip(),
            'Artista': artista.strip(),
            'Gravadora': gravadora.strip() if gravadora else 'Desconhecida',
            'Vezes Tocada': tocada
        })

    if not data:
        print('Nenhuma musica encontrada')
        return pd.DataFrame()

    df = pd.DataFrame(data)
    return df


In [7]:
df_musica_apos_2015 = criar_tabela_ate_2015(2022)

In [10]:
df_musica_apos_2015

,Música,Artista,Gravadora,Vezes Tocada
0,Termina comigo antes,Gusttavo Lima,Sony,74828
1,Bloqueado,Gusttavo Lima,Sony,68819
2,Abalo emocional,Luan Santana,Sony,61242
3,A maior saudade,Henrique e Juliano,Som Livre,58798
4,Erro planejado,"Luan Santana, Henrique e Juliano",Sony,57783
...,...,...,...,...
95,Melhor cilada,Théo e Gabriel,Independente,18228
96,Complicado,Akatu,Independente,18117
97,"Cabô, cabô",Patricia e Adriana,Independente,18090
98,Cavalo de pau,"Bruno e Denner, Gusttavo Lima",Independente,17897


## Funçao para criar tabela das musicas de 2014 ate 1917

In [12]:
def criar_tabela_anterior(ano):
    
    # definindo o headers e a url
    url_musica = f'https://maistocadas.mus.br/{ano}/'
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    # fazendo o http request e tratando erros (se a resposta nao for 200)
    try:
        response = requests.get(url_musica, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request falhou para o ano {ano}: {e}")

    # a lista de musicas destes anos estao de maneira diferente no html, agora é encontrada a div com a classe 'lista'
    # apos isto encontra a lista ordenada (<ol>) onde estao as musicas.
    soup = BeautifulSoup(response.text, 'html.parser')
    list_ordenada = soup.find('div', {'class': 'lista'}).find('ol')
    if list_ordenada is None:
        print('Lista ordenada nao encontrada')
        
    # processando as musicas encontradas, criando uma lista e passando um dicionario de cada musica para a lista com uma diferença
    # como é uma lista, é somente interar pelos <li>.
    data = []
    for li in list_ordenada.find_all('li'):
        # para cada item da lista, é extraido o seu texto, cortando qualquer espaço.
        text = li.get_text(strip=True)
        
        # conferir se contem -. Se conter, splita o texto em musica e artista, caso nao tenha, toda o texto sera o titulo da musica
        # e o artistia ficara como 'desconhecido'
        if ' – ' in text:
            musica, artista = text.split(' – ', 1)
        else:
            musica = text
            artista = 'Desconhecido'
        data.append({
            'Música': musica.strip(),
            'Artista': artista.strip(),
            'Gravadora': 'Desconhecido',
            'Vezes Tocada': 'Desconhecido'
        })

    if not data:
        print('Nenhuma musica encontrada')

    df = pd.DataFrame(data)
    return df

In [15]:
df_musica_antes_2015 = criar_tabela_anterior(1980)

In [16]:
df_musica_antes_2015

,Música,Artista,Gravadora,Vezes Tocada
0,Balancê,Gal Costa,Desconhecido,Desconhecido
1,Another Brick In The Wall,Pink Floyd,Desconhecido,Desconhecido
2,Crazy Little Thing Called Love,Queen,Desconhecido,Desconhecido
3,Momentos,Joanna,Desconhecido,Desconhecido
4,Menino do Rio,Baby Consuelo,Desconhecido,Desconhecido
...,...,...,...,...
95,Doce de pimenta,Olivia Byington,Desconhecido,Desconhecido
96,No night so long,Dionne Warwick,Desconhecido,Desconhecido
97,14 Bis,Nova manhã,Desconhecido,Desconhecido
98,The Second Time Around,Shalamar,Desconhecido,Desconhecido


## Funçao para vizualizar o ranking de cada ano, a partir de 2018 ate 1970

In [17]:
def vizualizar_ranking(ano):
    
    # definindo o headers e a url
    url_musica = f'https://maistocadas.mus.br/{ano}/'
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    
    # fazendo o http request e tratando erros (se a resposta nao for 200)
    try:
        response = requests.get(url_musica, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request falhou para o ano {ano}: {e}")

    # fazendo o parse do html, procurando a div class ranking.
    soup = BeautifulSoup(response.text, 'html.parser')
    ranking_div = soup.find('div', {'class': 'ranking'})
    if ranking_div is None:
        print('Nao encontrado o ranking.')

    data = []
    # interando sobre todos os itens da lista <li>.
    for li in ranking_div.find_all('li'):
        # para cada item da lista, é procurado a class posicao, para pegar seu texto, caso nao encontre a posicao, fica com Desconhecido.
        posicao = li.find('span', {'class': 'posicao'}).get_text(strip=True) if li.find('span', {'class': 'posicao'}) else 'Desconhecido'
        
        # aqui procuramos a tag <h3>, pegando o seu texto. Caso nao encontre, o genero sera Desconhecido.
        genero = li.find('h3').get_text(strip=True) if li.find('h3') else 'Desconhecido'
        data.append({
            'Posição': posicao,
            'Genero': genero
        })

    if not data:
        print('Nao foram encontrados generos.')

    df = pd.DataFrame(data)
    return df


In [18]:
df_ranking = vizualizar_ranking(1980)

In [19]:
df_ranking

,Posição,Genero
0,1º,MPB: 48 músicas (48%)
1,2º,Internacionais: 46 musicas
2,3º,Romântica: 4 músicas
3,4º,Pagode/Samba: 1 música


## Funçao para criar uma tabela com o ano e o genero mais ouvido.

In [21]:
def rankin_genero(ano):
    # definindo o headers
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}
    # fazendo o http request e tratando erros (se a resposta nao for 200)
    url_musica = f'https://maistocadas.mus.br/{ano}/'
    try:
        response = requests.get(url_musica, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request falhou para o ano {ano}: {e}")
        return None
    
    # fazendo o parse do html, procurando a div class ranking.
    soup = BeautifulSoup(response.text, 'html.parser')
    ranking_div = soup.find('div', {'class': 'ranking'})
    if ranking_div is None:
        print(f'Ranking não encontrado no ano {ano}')
        return None

    data = {}
    
    # interando sobre todos os itens da lista <li>.
    for li in ranking_div.find_all('li'):
        # para cada item da lista, é procurado a class posicao, para pegar seu texto, caso nao encontre a posicao, fica com Desconhecido.
        posicao = li.find('span', {'class': 'posicao'}).get_text(strip=True) if li.find('span', {'class': 'posicao'}) else 'Desconhecido'
        
        # aqui procuramos a tag <h3>, pegando o seu texto. Caso nao encontre, o genero sera Desconhecido.
        genero_info = li.find('h3').get_text(strip=True) if li.find('h3') else 'Desconhecido'
        
        # aqui juntamos o genero com sua respectiva posicao
        genero = genero_info.split(':')[0]
        data[genero] = posicao

    return data

In [23]:
data_total = {}
for ano in range(2022, 1969, -1):
    data_genero = rankin_genero(ano)
    if data_genero:
        data_total[ano] = data_genero
        
df_tabela_ranking = pd.DataFrame(data_total)

Ranking não encontrado no ano 2020
Ranking não encontrado no ano 2019


In [25]:
df_tabela_ranking

,2022,2021,2018,2017,2016,2015,2014,2013,2012,2011,...,1979,1978,1977,1976,1975,1974,1973,1972,1971,1970
Sertanejo,1º,1º,1º,1º,1º,1º,1º,1º,2º,2º,...,NaN,NaN,NaN,NaN,NaN,6º,6º,7º,6º,6º
Samba/Pagode,2º,3º,2º,NaN,3º,5º,4º,3º,3º,5º,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Forró/Pisadinha/Brega,3º,2º,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Internacionais,4º,4º,5º,4º,5º,2º,2º,2º,1º,1º,...,1º,1º,1º,1º,1º,1º,1º,1º,1º,1º
Pop nacional,NaN,5º,4º,3º,4º,3º,3º,4º,5º,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Forró,NaN,NaN,3º,2º,2º,6º,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6º,6º,8º
Funk,NaN,NaN,6º,NaN,6º,NaN,7º,7º,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MPB,NaN,NaN,NaN,5º,NaN,NaN,NaN,NaN,NaN,NaN,...,2º,2º,2º,2º,3º,3º,2º,2º,2º,2º
Gospel,NaN,NaN,NaN,NaN,NaN,4º,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pop-rock,NaN,NaN,NaN,NaN,NaN,NaN,5º,5º,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
